In [59]:
import os
os.chdir('/Coding/CVPR2025_abaw_framewise/')

In [93]:
import pandas as pd
import numpy as np
from sklearn.metrics import f1_score
from scipy.ndimage import gaussian_filter1d


res_path = "/Coding/CVPR2025_abaw_framewise/tools/output/0-7197-ep5.txt"
eval_path = "/Coding/CVPR2025_abaw_framewise/data_abaw/splits/val_relaxed.csv"



eval_df = pd.read_csv(eval_path)
res = pd.read_csv(res_path , names=["1","2","class_id","general_path"], sep=",", header=None)

true_labels = eval_df['class_id']
scores = res['class_id']

threshold = 0.577

average = "weighted"

predicted_labels = (scores > threshold).astype(int)

f1 = f1_score(true_labels, predicted_labels, average=average)

print(f"F1-Score: {f1:.4f}")

# Moving average smoothing filter function
def moving_average_filter(data, window_size):
    # Ensure data is a numpy array of floats
    data = np.array(data, dtype=float)
    kernel = np.ones(window_size, dtype=float) / window_size
    return np.convolve(data, kernel, mode='same')

# Initialize variables for optimization
best_f1 = 0
best_window = None

# Loop over possible window sizes (1 to 20)
for window in range(1, 200):
    # Apply the smoothing filter with the current window size
    smoothed_scores = moving_average_filter(scores, window)
    
    # Apply the fixed threshold to obtain binary predictions
    predicted_labels = ((1)*smoothed_scores >= threshold).astype(int)
    
    # Compute the F1-score using the specified averaging method
    current_f1 = f1_score(true_labels, predicted_labels, average=average)
    
    # Update the best F1-score and corresponding window size if improvement is found
    if current_f1 > best_f1:
        best_f1 = current_f1
        best_window = window

# Apply the smoothing filter with the optimal window size for final predictions
final_smoothed_scores = moving_average_filter(scores, best_window)
final_predictions = (final_smoothed_scores >= threshold).astype(int)

# Print the optimal window size and the maximized F1-score
print(f"Optimal window size: {best_window}")
print(f"Maximized F1-Score: {best_f1:.4f}")

F1-Score: 0.7250
Optimal window size: 60
Maximized F1-Score: 0.7279


In [106]:
# Gaussian smoothing filter function using scipy's gaussian_filter1d
def gaussian_smoothing_filter(data, sigma):
    return gaussian_filter1d(data.astype(float), sigma=sigma)

# Initialize variables for parameter optimization
best_f1 = 0
best_sigma = None

# Loop over possible sigma values (e.g., from 0.5 to 5.0 with step 0.5)
for sigma in np.arange(0.5, 5.5, 0.1):
    # Apply the Gaussian smoothing filter with the current sigma
    smoothed_scores = gaussian_smoothing_filter(scores, sigma)
    
    # Apply the fixed threshold to obtain binary predictions
    predicted_labels = (smoothed_scores >= threshold).astype(int)
    
    # Compute the F1-score using the specified averaging method
    current_f1 = f1_score(true_labels, predicted_labels, average=average)
    
    # Update the best F1-score and corresponding sigma if improvement is found
    if current_f1 > best_f1:
        best_f1 = current_f1
        best_sigma = sigma

# Apply the smoothing filter with the optimal sigma for final predictions
final_smoothed_scores = gaussian_smoothing_filter(scores, best_sigma)
final_predictions = (final_smoothed_scores >= threshold).astype(int)

# Print the optimal sigma and the maximized F1-score
print(f"Optimal sigma: {best_sigma}")
print(f"Maximized F1-Score: {best_f1:.4f}")

Optimal sigma: 3.7999999999999994
Maximized F1-Score: 0.7264


In [108]:
from scipy.ndimage import uniform_filter1d
import numpy as np
from sklearn.metrics import f1_score

# Uniform Smoothing Filter Funktion
def uniform_smoothing_filter(data, size):
    return uniform_filter1d(data.astype(float), size=size)

# Initialisierung der Variablen für die Optimierung
best_f1_uniform = 0
best_size_uniform = None

# Schleife über mögliche Fenstergrößen (z. B. von 3 bis 51 in Schritten von 2)
for size in range(3, 70, 2):
    # Filterung der Scores mit der aktuellen Fenstergröße
    smoothed_scores = uniform_smoothing_filter(scores, size)
    
    # Erzeugen der binären Vorhersagen basierend auf einem festen Schwellenwert
    predicted_labels = (smoothed_scores >= threshold).astype(int)
    
    # Berechnung des F1-Scores
    current_f1 = f1_score(true_labels, predicted_labels, average=average)
    
    # Update der besten Parameter, falls eine Verbesserung erzielt wurde
    if current_f1 > best_f1_uniform:
        best_f1_uniform = current_f1
        best_size_uniform = size

# Ausgabe der optimalen Fenstergröße und des maximierten F1-Scores für uniform_filter1d
print(f"Optimaler Fenstergröße für uniform_filter1d: {best_size_uniform}")
print(f"Maximierter F1-Score für uniform_filter1d: {best_f1_uniform:.4f}")


Optimaler Fenstergröße für uniform_filter1d: 45
Maximierter F1-Score für uniform_filter1d: 0.7277


In [109]:
from scipy.ndimage import maximum_filter1d
import numpy as np
from sklearn.metrics import f1_score

# Maximum Smoothing Filter Funktion
def maximum_smoothing_filter(data, size):
    return maximum_filter1d(data.astype(float), size=size)

# Initialisierung der Variablen für die Optimierung
best_f1_maximum = 0
best_size_maximum = None

# Schleife über mögliche Fenstergrößen (z. B. von 3 bis 51 in Schritten von 2)
for size in range(3, 52, 2):
    # Filterung der Scores mit der aktuellen Fenstergröße
    smoothed_scores = maximum_smoothing_filter(scores, size)
    
    # Erzeugen der binären Vorhersagen basierend auf einem festen Schwellenwert
    predicted_labels = (smoothed_scores >= threshold).astype(int)
    
    # Berechnung des F1-Scores
    current_f1 = f1_score(true_labels, predicted_labels, average=average)
    
    # Update der besten Parameter, falls eine Verbesserung erzielt wurde
    if current_f1 > best_f1_maximum:
        best_f1_maximum = current_f1
        best_size_maximum = size

# Ausgabe der optimalen Fenstergröße und des maximierten F1-Scores für maximum_filter1d
print(f"Optimaler Fenstergröße für maximum_filter1d: {best_size_maximum}")
print(f"Maximierter F1-Score für maximum_filter1d: {best_f1_maximum:.4f}")


Optimaler Fenstergröße für maximum_filter1d: 3
Maximierter F1-Score für maximum_filter1d: 0.7242


In [105]:
best_f1 = f1_score(true_labels, predicted_labels, average=average)
best_zeroes_to_fill = 0

def optimize_zeroes_to_fill(res, zeroes_to_fill):
    adjusted_predictions = predicted_labels.copy()
    last_path = None
    zero_count = 0

    i = 0
    n = len(res)
    while i < n:
        current_path = res["general_path"].iloc[i]

        if last_path != current_path:
            zero_count = 0
            last_path = current_path

        if adjusted_predictions[i] == 0:
            zero_count += 1

        if adjusted_predictions[i] == 1:
            if zero_count <= zeroes_to_fill:
                start_index = i - zero_count  # first index of current zero sequence
                
                # Replace zeros after the allowed threshold in the current sequence
                for k in range(start_index, i + 1):
                    adjusted_predictions[k] = 1
                
                zero_count = 0
        
        i += 1
        
    return adjusted_predictions

for zeroes in range(1, 50):  
    new_predictions = optimize_zeroes_to_fill(res, zeroes)
    current_f1 = f1_score(true_labels, new_predictions, average=average)

    print(f"zeroes_to_fill: {zeroes}")
    print(f"F1-Score: {current_f1:.4f}")

    if current_f1 > best_f1:
        best_f1 = current_f1
        best_zeroes_to_fill = zeroes

final_predictions = optimize_zeroes_to_fill(res, best_zeroes_to_fill)

print(f"Optimal zeroes_to_fill: {best_zeroes_to_fill}")
print(f"Maximized F1-Score: {best_f1:.4f}")

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0

In [90]:
    
arr = [0,0,0,1,1,1,1,0,1,1,1,0,0,0,0,0,1,1,0,0,0,0]
zeroes_to_fill = 3

i = 0
n = 10
zero_count = 0
while i < n:

    if arr[i] == 0:
        zero_count += 1

    if arr[i] == 1:
        if zero_count <= zeroes_to_fill:
            start_index = i - zero_count   # first index of current zero sequence
        
            # Replace zeros after the allowed threshold in the current sequence
            for k in range(start_index, i + 1):
                arr[k] = 1

        zero_count = 0
    
    i += 1

print(arr)

[0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0]


In [74]:
predicted_labels = (scores > threshold).astype(int)
best_f1 = f1_score(true_labels, predicted_labels, average=average)

print(best_f1)

best_zeroes_to_fill = 0
best_thresh = 0
best_part_tresh = 0

def optimize_zeroes_to_fill(scores, res, zeroes_to_fill, best_f1, best_thresh, true_labels):
    for thresh_search in np.arange(0.35, 0.7, 0.01):
        adjusted_predictions = scores.copy()
        last_path = None
        zero_count = 0
        for i in range(len(res)):
            current_path = res["general_path"].iloc[i]

            if last_path != current_path:
                zero_count = 0
                last_path = current_path

            if adjusted_predictions[i] < thresh_search:
                zero_count += 1
            else:
                if zero_count > 0 and zero_count <= zeroes_to_fill:
                    adjusted_predictions[i - zero_count : i] = 1
                zero_count = 0  

        new_predictions = (scores > threshold).astype(int)
        f1 = f1_score(true_labels, new_predictions, average=average)
        
        if f1 > best_f1:
            best_f1 = f1
            best_thresh = thresh_search

    return adjusted_predictions, best_thresh

for cur_tresh in np.arange(0.35, 0.7, 0.01):

    for zeroes in range(1, 50):  
        new_predictions, tresh = optimize_zeroes_to_fill(scores, res, zeroes, best_f1, best_thresh=best_part_tresh, true_labels=true_labels)
        current_f1 = f1_score(true_labels, new_predictions, average=average)

        print(f"f1: {current_f1} - zeroes: {zeroes} - search_threshold: {best_thresh} - threshold: {cur_tresh}")

        if current_f1 > best_f1:
            best_f1 = current_f1
            best_zeroes_to_fill = zeroes
            best_part_tresh = tresh
            best_thresh = cur_tresh

    final_predictions = optimize_zeroes_to_fill(res, best_zeroes_to_fill)

print(f"Optimal zeroes_to_fill: {best_zeroes_to_fill}")
print(f"Maximized F1-Score: {best_f1:.4f}")

0.7250388483323572


ValueError: Classification metrics can't handle a mix of binary and continuous targets

In [63]:
# Gaussian smoothing filter function using scipy's gaussian_filter1d
def gaussian_smoothing_filter(data, sigma):
    return gaussian_filter1d(data.astype(float), sigma=sigma)

# Initialize variables for parameter optimization
best_f1 = 0
best_sigma = None

# Loop over possible sigma values (e.g., from 0.5 to 5.0 with step 0.5)
for sigma in np.arange(0.5, 5.5, 0.1):
    # Apply the Gaussian smoothing filter with the current sigma
    smoothed_scores = gaussian_smoothing_filter(scores, sigma)
    
    # Apply the fixed threshold to obtain binary predictions
    predicted_labels = (smoothed_scores >= threshold).astype(int)
    
    # Compute the F1-score using the specified averaging method
    current_f1 = f1_score(true_labels, predicted_labels, average=average)
    
    # Update the best F1-score and corresponding sigma if improvement is found
    if current_f1 > best_f1:
        best_f1 = current_f1
        best_sigma = sigma

# Apply the smoothing filter with the optimal sigma for final predictions
final_smoothed_scores = gaussian_smoothing_filter(scores, best_sigma)
final_predictions = (final_smoothed_scores >= thres).astype(int)

# Print the optimal sigma and the maximized F1-score
print(f"Optimal sigma: {best_sigma}")
print(f"Maximized F1-Score: {best_f1:.4f}")

Optimal sigma: 3.7999999999999994
Maximized F1-Score: 0.7264


In [66]:
# Initial binary prediction based on the threshold
initial_pred = (scores >= threshold).astype(int)

def fill_short_gaps(binary_array, max_gap):
    """
    Fill gaps (zeros) that are surrounded by ones if the gap length is
    less than or equal to max_gap.
    """
    filled = binary_array.copy()
    n = len(filled)
    i = 0
    while i < n:
        if filled[i] == 0:
            start = i
            while i < n and filled[i] == 0:
                i += 1
            gap_length = i - start
            # Only fill if gap is short and flanked by ones
            if start > 0 and i < n and gap_length <= max_gap:
                filled[start:i] = 1
        else:
            i += 1
    return filled

def average_segment_length(binary_array):
    """
    Calculate the average length of contiguous segments where the value is 1.
    """
    segments = []
    count = 0
    for value in binary_array:
        if value == 1:
            count += 1
        elif count > 0:
            segments.append(count)
            count = 0
    if count > 0:
        segments.append(count)
    return np.mean(segments) if segments else 0

def optimize_gap_filling(binary_array, true_labels, target_length=112.2,
                         aggressiveness=1.0, max_gap_search=50, average_method='weighted'):
    """
    Optimize the maximum gap length parameter that:
      1. Adjusts the average segment length toward target_length.
      2. Maximizes the F1-score (comparing the filled binary prediction with true_labels).

    The aggressiveness parameter scales the search range for max_gap.
    """
    best_f1 = 0
    best_gap = None
    best_filled = binary_array.copy()
    best_avg_diff = None

    # Search candidate gap sizes from 1 up to (max_gap_search * aggressiveness)
    for gap in range(1, int(max_gap_search * aggressiveness) + 1):
        filled = fill_short_gaps(binary_array, gap)
        avg_len = average_segment_length(filled)
        diff = abs(avg_len - target_length)
        current_f1 = f1_score(true_labels, filled, average=average_method)

        # Primary objective: maximize F1-score.
        # (We also report the difference from the target average length.)
        if current_f1 > best_f1:
            best_f1 = current_f1
            best_gap = gap
            best_filled = filled.copy()
            best_avg_diff = diff

    return best_gap, best_filled, best_f1, best_avg_diff

# Set the aggressiveness parameter (higher -> more aggressive gap filling, testing larger gaps)
aggressiveness = 1.0 * 20

# Optimize gap filling to maximize F1 while trying to approach the target segment length of 112.2 samples
optimal_gap, final_predictions, best_f1, avg_diff = optimize_gap_filling(
    initial_pred, true_labels,
    target_length=112.2,
    aggressiveness=aggressiveness,
    max_gap_search=75,
    average_method="weighted"
)

final_avg_length = average_segment_length(final_predictions)

# Print the optimal gap parameter, maximized F1-score, and average segment length details
print(f"Optimal gap to fill (max_gap): {optimal_gap}")
print(f"Maximized F1-Score: {best_f1:.4f}")
print(f"Final average segment length: {final_avg_length:.2f} samples")
print(f"Difference from target average length (112.2): {avg_diff:.2f} samples")

KeyboardInterrupt: 

In [53]:
initial_predictions = (scores >= threshold).astype(int)

def fill_short_gaps(binary_array, max_gap):
    """
    Fills short gaps (consecutive zeros) in a binary array.
    A gap is filled if its length is less than or equal to max_gap and is flanked by ones.
    
    Parameters:
        binary_array: numpy array of binary values (0 or 1)
        max_gap: maximum gap length to fill
        
    Returns:
        A new binary array with short gaps filled.
    """
    filled = binary_array.copy()
    n = len(filled)
    i = 0
    while i < n:
        if filled[i] == 0:
            start = i
            # Count the length of the zero gap
            while i < n and filled[i] == 0:
                i += 1
            gap_length = i - start
            # Fill the gap if it is flanked by ones and is small enough
            if start > 0 and i < n and gap_length <= max_gap:
                filled[start:i] = 1
        else:
            i += 1
    return filled

# Optimization loop: find the candidate max_gap that maximizes the F1 score.
best_f1 = 0
best_gap = None
best_filled_predictions = initial_predictions.copy()

# Try candidate gap values from 1 up to 50 (adjust range as needed)
for candidate_gap in range(1, 100):
    filled_predictions = fill_short_gaps(initial_predictions, candidate_gap)
    current_f1 = f1_score(true_labels, filled_predictions)
    
    if current_f1 > best_f1:
        best_f1 = current_f1
        best_gap = candidate_gap
        best_filled_predictions = filled_predictions.copy()

# Print the optimal gap parameter and the maximized F1 score
print(f"Optimal gap parameter (max_gap): {best_gap}")
print(f"Maximized F1 Score: {best_f1:.4f}")

Optimal gap parameter (max_gap): 66
Maximized F1 Score: 0.3549
